# 1 概念
核心思想是分布假说（Distributional Hypothesis）：

> you shall know a word by the company it keeps

利用 word2Vec核心模型Skip-Gram，我们有个中心词 $w_c$，我们要预测它周围的上下文词 $w_o$，目标是最大化似然函数 $L(\theta)$:

$$L(\theta)=\prod_{t=1}^{T}\prod_{-m \le j \le m, j \ne 0}P(w_{t+j}|w_t;\theta)$$

即最小化 负对数似然（Loss Function）：

$$J(\theta)=-\frac{1}{T} \sum_{t=1}^{T} \sum_{-m \le j \le m, j \ne 0}P(w_{t+j}|w_t;\theta)$$

**核心公式** softmax 预测概率：
为了计算 $P(w_o|w_c)$，我们需要两套向量：

1. $v_w$：当$w$是中心词时的向量
2. $u_w$: 当$w$是上下文词时的向量

$$P(w_o|w_c)=\frac{\exp(u_{o}^T v_{c})}{\sum_{w=1}^{V} \exp(u_{w}^T v_{c})}$$

- 分子：${u_o}^T{v_c}$是点积，向量越相似，点积越大，概率越大
- 分母：归一化项（所有词汇表中单词的指数和），确保概率为1



## 1.1 负对数似然(Negative Log-Liklihood, NLL)

这是我们训练神经网络时，用来衡量“模型有多蠢”的一把尺子。拆分开三层来理解：

**1. 什么是 Likelihood 似然**

似然：“模型猜对真值的概率”

中心词 King，上下文应该是Queen。

模型预测（Softmax输出）：
- Queen: 0.9（很有信心）-> Likelihood = 0.9（好模型）
- Queen：0.1（瞎猜）-> Likelihood = 0.1（烂模型）

目标：最大化似然，概率越高越好。

**2. 为什么要加Log（对数）**

实际训练中，通常是一次训练一批数据，假设一批数据有3个样本，预测正确概率分别是 0.9, 0.8, 0.9

联合概率是乘积关系：$L=0.9 \times 0.8 \times 0.9 = 0.648$

但如果序列很长 $0.9^{1000}$ 就会变成一个无限接近于0的数，计算机会直接变成0，而且乘法求导难算。

解决办法：取对数（Log），$\log(a \times b) = \log(a) + \log(b)$，无论怎么加，数值都很稳定，而且容易求导。

**3.为什么要加负号 Negative**

- Likelihood/Log-Likelihood 是得分，所以越高越好
- Loss Function 是错误/代价，所以越低越好

加一个负号即可。

$$Loss = - \log(P(\text{正确的词}|\text{当前输入}))$$

## 1.2 Softmax 函数

假设现在中心词是 King，我们要预测它旁边的词是谁，词表里面有三个候选人：Queen，Dog，Apple。

神经网络内部通过向量点积 $(u^T v)$，给三个候选人打出相似度得分(Logits)：
- Queue 得分 3.0
- Dog 得分 1.0
- Apple 得分 -1.0

但是概率不能为负数，现在三个和也不等于1，为了解决这些问题，并且拉大差距，对得分取指数：
- Queue ：$e^3.0 \approx 20.1$
- Dog : $e^1.0 \approx 2.7$
- Apple : $e^{-1.0} \approx 0.37$

之后为了让概率总和100，就需要每个概率除以分母，就是上面的公式：

$$P(w_o|w_c)=\frac{\exp(u_{o}^T v_{c})}{\sum_{w=1}^{V} \exp(u_{w}^T v_{c})}$$

## 1.3 $e^x$的核心好处

1. 马太效应（winner takes all）：指数函数增长极快，softmax会极度放大得分最高的那个词，让模型在预测时更加果断
2. 可导性（Differentiability）：$e^x$的导数还是$e^x$，在反响传播计算梯度的时候，数学性质丝滑

# 2 Coding
## 2.1 Numpy 手搓梯度流向
手动计算 Softmax的梯度，梯度推导，Loss对预测的$z$的导数：
$$\frac{\partial J}{\partial z}=\text{preds}-y{\text{true}}$$

Skip-gram模型中，每个词有2个身份：

1. 做为中心词：当他去预测别人时，用W1中的向量
2. 作为上下文：当别人用来预测它时，它用W2中的向量

- W1(vocab_size, embed_dim)：
    - 每一行i存储第i个词的Input Vector($v_i$)
    - 当输入是 One-Hot（第k位是1）时，OneHot @ W1实际上就是取出W1的第k行
- W2(embed_dim, vocab_size):
    - 为了方便点积，所以进行转置
    - 每列j存储了第j个词的Output vector($u_j$)

In [ ]:
import numpy as np

class Word2VecNumpy:
    def __init__(self, vocab_size, embed_dim, learning_rate=0.01):
        self.vocab_size = vocab_size
        self.lr = learning_rate
        self.W1 = np.random.randn(vocab_size, embed_dim) * 0.01 # 中心词矩阵
        self.W2 = np.random.rand(embed_dim, vocab_size) * 0.01 # 上下文矩阵
    
    def softmax(self, x):
        # 减去最大值防止溢出
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
     
    def forward_backward(self, center_idx, context_idx):
        """
        center_idx: int, 中心词在词表中的索引
        context_idx: int, 上下文词在词表中的索引
        """
        # --- forward ---
        # 1. 提取中心词向量 （相当于 One-Hot * W1）
        h = self.W1[center_idx]  # shape: (embed_dim,)

        # 2. 计算与所有词的相似度得分(Logits)
        z = np.dot(self.W2.T, h)  # shape: (vocab_size,)

        # 3. 计算预测概率 softmax
        y_pred = self.softmax(z)  # shape: (vocab_size,)

        # --- backward ---
        # 1. 计算输出层的误差
        e = y_pred.copy()  # shape: (vocab_size,)
        e[context_idx] -= 1 # 只在正确答案的位置-1，其余不变

        # 2. 计算W2梯度
        dW2 = np.outer(h, e)  # shape: (embed_dim, vocab_size)

        # 3. 计算W1梯度
        dW1_row = np.dot(self.W2, e)  # shape: (embed_dim,)

        # update
        self.W2 -= self.lr * dW2
        self.W1[center_idx] -= self.lr * dW1_row

        loss = -np.log(y_pred[context_idx] + 1e-10)  # 计算交叉熵损失
        return loss
    
# -- test sample --
vocab_size = 10
embed = 3
model = Word2VecNumpy(vocab_size, embed)

print("Training on a single sample (center_idx=2, context_idx=3)")
for i in range(1000):
    loss = model.forward_backward(center_idx=2, context_idx=3)
    if (i+1) % 200 == 0:
        print(f"Iteration {i+1}, Loss: {loss:.4f}")
print("\nResult vectors:")
print(f"Vector for word 2: {model.W1[2]}")

Training on a single sample (center_idx=2, context_idx=3)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3)
(3,)
(10, 3